In [ ]:
'''

'''

In [1]:
import torch 
from torch import nn


不含模型参数的自定义层

In [2]:
class center(nn.Module):
    def __init__(self,**kwargs):
        super(center,self).__init__(**kwargs)
    def forward(self,x):
        return x-x.mean()
        

In [4]:
layer = center()
layer(torch.tensor([1,2,3,4,5],dtype=torch.float))

tensor([-2., -1.,  0.,  1.,  2.])

In [5]:
net=nn.Sequential(nn.Linear(8,128),center())
y=net(torch.rand(4,8))
y.mean().item()

-4.6566128730773926e-09

含模型参数的自定义层

In [11]:
class mylistdense(nn.Module):
    def __init__(self):
        super(mylistdense,self).__init__()
        self.param = nn.ParameterList([nn.Parameter(torch.randn(4,4)) for i in range(3)])
        self.param.append(nn.Parameter(torch.randn(4,1)))
    def forward(self,x):
        for i in range (len(self.param)):
            x = torch.mm(x, self.param[i])
        return x
net = mylistdense()
print(net)

mylistdense(
  (param): ParameterList(
      (0): Parameter containing: [torch.FloatTensor of size 4x4]
      (1): Parameter containing: [torch.FloatTensor of size 4x4]
      (2): Parameter containing: [torch.FloatTensor of size 4x4]
      (3): Parameter containing: [torch.FloatTensor of size 4x1]
  )
)


In [16]:
class mydensedict(nn.Module):
    def __init__(self):
        super(mydensedict,self).__init__()
        self.param = nn.ParameterDict({
            'linear1':nn.Parameter(torch.rand(4,4)),
            'linear2':nn.Parameter(torch.rand(4,1))
        })
        self.param.update({'linear3':nn.Parameter(torch.rand(4,2))})
        
    def forward(self,x,choice='linear1'):
        return torch.mm(x,self.param[choice])

In [17]:
net=mydensedict()
print(net)

mydensedict(
  (param): ParameterDict(
      (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
      (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
      (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
  )
)


In [18]:
x = torch.ones(1,4)
print(net(x,'linear1'))
print(net(x,'linear2'))
print(net(x,'linear3'))

tensor([[1.7676, 2.4045, 1.7296, 1.4598]], grad_fn=<MmBackward>)
tensor([[2.8670]], grad_fn=<MmBackward>)
tensor([[2.3992, 1.6821]], grad_fn=<MmBackward>)


In [21]:
net = nn.Sequential(mydensedict(), mylistdense())
print(net)
print(net(x))

Sequential(
  (0): mydensedict(
    (param): ParameterDict(
        (linear1): Parameter containing: [torch.FloatTensor of size 4x4]
        (linear2): Parameter containing: [torch.FloatTensor of size 4x1]
        (linear3): Parameter containing: [torch.FloatTensor of size 4x2]
    )
  )
  (1): mylistdense(
    (param): ParameterList(
        (0): Parameter containing: [torch.FloatTensor of size 4x4]
        (1): Parameter containing: [torch.FloatTensor of size 4x4]
        (2): Parameter containing: [torch.FloatTensor of size 4x4]
        (3): Parameter containing: [torch.FloatTensor of size 4x1]
    )
  )
)
tensor([[-21.2719]], grad_fn=<MmBackward>)
